XGboost算法

　-　XGBoost是GBDT算法的一种改进，是一种常用的有监督集成学习算法；是一种伸缩性强、便捷的可并行构建模型的GradientBoosting算法。

　-　原理是：在GBDT目标函数的基础上加入惩罚项，如下图绿框。通过限制树模型的叶子节点的个数和叶子节点的值来降低模型复杂度，从而防止过拟合，二分之一是为了求导方便。t是树的棵数，obj为损失函数

　-　大体步骤：防止过拟合，二阶泰勒展开公式计算，给出了新的树的划分标准，用的是损失函数的增量。

　-　目的：找到第t颗树是如何搭建的

优点
   - 精度更高：XGBoost 对损失函数进行了二阶泰勒展开

   - 灵活性更强：XGBoost 不仅支持 CART 还支持线性分类器，使用线性分类器的 XGBoost 相当于带 L1 和 L2 正则化项的逻辑斯蒂回归（分类问题）或者线性回归（回归问题）。此外，XGBoost 工具支持自定义损失函数，只需函数支持一阶和二阶求导；

   - 正则化：XGBoost 在目标函数中加入了正则项，用于控制模型的复杂度。正则项里包含了树的叶子节点个数、叶子节点权重的 L2 范式。正则项降低了模型的方差，使学习出来的模型更加简单，有助于防止过拟合.

   - Shrinkage（缩减）：相当于学习速率。XGBoost 在进行完一次迭代后，会将叶子节点的权重乘上该系数，主要是为了削弱每棵树的影响，让后面有更大的学习空间。

   - 列抽样：XGBoost 借鉴了随机森林的做法，支持列抽样，不仅能降低过拟合，还能减少计算。

   - 缺失值处理：对于特征的值有缺失的样本，XGBoost 采用的稀疏感知算法可以自动学习出它的分裂方向；

   - XGBoost工具支持并行：XGBoost的并行是在特征粒度上的。各个特征的增益计算可以开多线程进行。

   - 可并行的近似算法：XGBoost提出了一种可并行的近似算法，用于高效地生成候选的分割点。


缺点
  -  虽然利用预排序和近似算法可以降低寻找最佳分裂点的计算量，但在节点分裂过程中仍需要遍历数据集；

   - 预排序过程的空间复杂度过高，不仅需要存储特征值，还需要存储特征对应样本的梯度统计值的索引，相当于消耗了两倍的内存。

算法过程
- 读取数据
- 修复数据并删除异常值
- shops/cats/items对象和特征预处理
-  在训练集中创建矩阵存储每个月的商店-商品对
-获取训练集中每个商店-商品对的月销售额并合并到矩阵中
- 按 (0,20) 剪裁 item_cnt_month
- 将测试集合并到矩阵中，第34个月用零值填充NaN数据
- 将shops/cats/items合并到矩阵中
- 添加滞后特征
- 添加平均编码特征
- 添加价格趋势特征
- 添加月份
- 添加天数
- 添加距离最近一次销售时间和新商品销售时间
- 删除第一年和无法被测试集计算的列
- 选择最佳特征
- 设置验证策略是测试集第34个月，验证集第33个月，训练集13月到33月
- 为测试集拟合模型，预测，裁剪获取目标值

这里我觉得应该是整体代码中最为核心的部分，我理解是这一种时间滑窗的操作，通过时间平移，
创建时间滞后变量——也就是上一个时间的数据作为当前时间戳的预测变量来预测目标值，
可以理解成一种隐式的用过去时间数据来预测当前数据目标值。后面所有的操作都是基于此的。

In [ ]:
def lag_feature(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df
matrix = lag_feature(matrix, [1,2,3,6,12], 'item_cnt_month')



注意点

- 给出的是每日的销售量统计，但是预测的是一个月的销售量
- 注意相比上个月，下个月可能会出现一些新的商品，也就是说，新的时间中会有历史数据中不曾出现过的样本数据，对预测造成一定的影响；
- 预测目标值被限定了在[0,20]，在最终预测之后，要做一次裁剪
- 本题测试集什么特征变量都没有
- Revenue trend features
1.validation set和test set都会出现train set所没有的新商品，借助商店revenue趋势特征可以帮助算法预测新商品在现有店铺的销量。
- new item features
1. 'item_shop_last_sale'和'item_last_sale'用以纪录距离最近一次销售之间隔了几个月，通过它可以和之前月份的数据建立关联。
2.'item_shop_first_sale'和'item_first_sale'则是用于表示新商品的特征。
- 代码里对数据的类型进行了降低，一般来说，我们不去管数据的精度保持默认情况就行，但较小的数据精度，在大规模数据处理中具有很重要的作用，可以加快数据处理以及节省内存